# LEGO Bricks Detection - YOLOv8 Multiclass Training

This notebook trains a YOLOv8 model for LEGO brick detection using Kaggle T4 GPUs.

In [ ]:
!pip install ultralytics albumentations

In [ ]:
# Clone repository
!git clone https://github.com/MiguelDiLalla/LEGO_Bricks_ML_Vision.git
%cd LEGO_Bricks_ML_Vision

In [ ]:
import os
import shutil
import random
from pathlib import Path
import json

# Create cache directory structure
cache_dir = Path('cache')
for split in ['train', 'val', 'test']:
    (cache_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (cache_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Get all image files
image_files = list(Path('data/images').glob('*.jpg'))
random.shuffle(image_files)

# Split ratios
train_ratio, val_ratio = 0.7, 0.2
n_images = len(image_files)
n_train = int(n_images * train_ratio)
n_val = int(n_images * val_ratio)

# Split and copy files
splits = {
    'train': image_files[:n_train],
    'val': image_files[n_train:n_train + n_val],
    'test': image_files[n_train + n_val:]
}

for split, files in splits.items():
    for img_path in files:
        # Copy image
        shutil.copy2(img_path, cache_dir / split / 'images' / img_path.name)
        # Copy corresponding label
        label_path = Path('data/labels') / f'{img_path.stem}.txt'
        if label_path.exists():
            shutil.copy2(label_path, cache_dir / split / 'labels' / label_path.name)

print(f'Split complete: {len(splits["train"])} train, {len(splits["val"])} val, {len(splits["test"])} test')

In [ ]:
# Load class information from metadata
with open('data/labels/batch_inference_metadata.json', 'r') as file:
    metadata = json.load(file)
classes_dict = metadata['config']['classes']

# Create dataset.yaml with actual classes
yaml_content = f'''
path: {cache_dir.absolute()}  # dataset root dir
train: train/images  # train images
val: val/images  # val images
test: test/images  # test images

# Classes
names:
'''

# Add class definitions dynamically from metadata
for class_id, class_name in classes_dict.items():
    yaml_content += f'  {class_id}: {class_name}\n'

with open(cache_dir / 'dataset.yaml', 'w') as f:
    f.write(yaml_content)

print('Created dataset.yaml file with', len(classes_dict), 'classes')

In [ ]:
import albumentations as A
from ultralytics import YOLO

# Define augmentation pipeline
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Flip(p=0.5),
    A.OneOf([
        A.GaussNoise(p=1),
        A.GaussianBlur(p=1),
    ], p=0.3),
])

In [ ]:
# Load base model
model = YOLO('yolov8n.pt')

# Training configuration
model.train(
    data='cache/dataset.yaml',  # Updated path to data config file
    epochs=100,
    imgsz=640,
    batch=16,
    device='0,1',  # Use both T4 GPUs
    workers=8,
    amp=True,    # Mixed precision training
    augment=True,
    val=True,
    project='results',  # Save results to this directory
    name='lego_multiclass',
    save=True,
    cache=True,  # Cache images for faster training
    # Additional hyperparameters
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5
)

In [ ]:
# Validate the trained model
metrics = model.val()

In [ ]:
import shutil
from datetime import datetime

# Create timestamped zip filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'lego_training_results_{timestamp}.zip'

# Create zip file of results directory
shutil.make_archive(
    base_name=zip_filename.replace('.zip', ''),
    format='zip',
    root_dir='results'
)

print(f'Results saved to {zip_filename}')